In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > output.log 2>&1 &
!ollama pull phi4

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.



In [ ]:
# Install required libraries
!pip install deepgram-sdk pydub requests

# Import libraries
import os
import base64
import requests
from deepgram import DeepgramClient, PrerecordedOptions, SpeakOptions
from IPython.display import Audio, Javascript, display
from google.colab import output, files
from pydub import AudioSegment
import io
import time
import json

# Set up Deepgram client
API_KEY = '2e3d8166e27ee93b7ce2c3bd78a3beaa0f432d0e'  # Your Deepgram API key
deepgram = DeepgramClient(API_KEY)

# JavaScript code for recording audio
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader();
  reader.onloadend = () => resolve(reader.result);
  reader.readAsDataURL(blob);
});
var record = time => new Promise(async resolve => {
  try {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    recorder = new MediaRecorder(stream);
    chunks = [];
    recorder.ondataavailable = e => chunks.push(e.data);
    recorder.start();
    await sleep(time);
    recorder.onstop = async () => {
      blob = new Blob(chunks, { type: 'audio/wav' });
      text = await b2text(blob);
      resolve(text);
    };
    recorder.stop();
  } catch (err) {
    console.error('Error accessing microphone:', err);
    resolve('error: ' + err.message);
  }
});
"""

def record(seconds=5, filename='recorded_audio.wav'):
    print(f"Attempting to record audio for {seconds} seconds...")
    print("Please ensure your browser has microphone access enabled.")

    try:
        display(Javascript('console.log("JavaScript is running");'))
        display(Javascript(RECORD))
        js_result = output.eval_js(f'record({seconds * 1000})')

        if js_result.startswith('error:'):
            print(f"Recording failed: {js_result}")
            return upload_audio()

        audio_data = js_result.split(',')[1]
        audio_bytes = base64.b64decode(audio_data)

        with open(filename, 'wb') as f:
            f.write(audio_bytes)

        print(f"Audio successfully saved as {filename}")
        return filename

    except Exception as e:
        print(f"Error during recording: {e}")
        print("Falling back to file upload...")
        return upload_audio()

def upload_audio():
    print("Please upload an audio file (WAV or MP3) to transcribe.")
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded.")
        return None
    audio_file = list(uploaded.keys())[0]
    print(f"Uploaded audio file: {audio_file}")
    return audio_file

def transcribe_audio(audio_file):
    if not audio_file:
        print("No audio file to transcribe.")
        return None
    print("Transcribing audio...")
    try:
        with open(audio_file, 'rb') as f:
            audio_data = f.read()

        # Configure STT options
        options = PrerecordedOptions(
            model="nova-2",
            language="en",
            smart_format=True,
            punctuate=True
        )

        # Transcribe using Deepgram
        response = deepgram.listen.prerecorded.v("1").transcribe_file(
            {"buffer": audio_data},
            options
        )

        text = response['results']['channels'][0]['alternatives'][0]['transcript']
        if not text:
            print("No transcription detected.")
            return None
        print(f"Transcribed text: {text}")
        return text
    except Exception as e:
        print(f"STT error: {e}")
        return None

def text_to_speech(text, output_file="output_audio.mp3"):
    if not text:
        print("No text to convert to speech.")
        return None
    print("Generating speech from text...")
    try:
        # Configure TTS options
        options = SpeakOptions(
            model="aura-asteria-en",  # English voice
            encoding="mp3"
        )

        # Generate speech using Deepgram
        response = deepgram.speak.rest.v("1").save(
            output_file,
            {"text": text},
            options
        )

        print(f"Generated audio saved as {output_file}")
        return output_file
    except Exception as e:
        print(f"TTS error: {e}")
        return None

def main():
    # Record or upload audio
    audio_file = record(seconds=5)

    # Transcribe the audio
    transcribed_text = transcribe_audio(audio_file)

    # Convert transcribed text to speech
    output_audio = text_to_speech(transcribed_text)

    # Play and download the generated audio
    if output_audio:
        print("Playing generated audio...")
        display(Audio(filename=output_audio, autoplay=True))
        print("Download the files:")
        if audio_file:
            files.download(audio_file)
        files.download(output_audio)

# Run the workflow
if __name__ == "__main__":
    main()

Attempting to record audio for 5 seconds...
Please ensure your browser has microphone access enabled.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Audio successfully saved as recorded_audio.wav
Transcribing audio...


<ipython-input-12-c3544a8ffff2>:104: UnsupportedWarning: prerecorded is unsupported as of 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  response = deepgram.listen.prerecorded.v("1").transcribe_file(


...

Download the files:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install ollama langchain

In [ ]:
# Install dependencies
!pip install -q deepgram-sdk pydub requests langchain langchain-community langchain-ollama openai

# Imports
import os
import base64
from deepgram import DeepgramClient, PrerecordedOptions, SpeakOptions
from IPython.display import Audio, Javascript, display
from google.colab import output, files
from pydub import AudioSegment
import io
import time
import json

# New LLM import
from langchain_ollama import OllamaLLM

# --- SETUP ---

# Deepgram API key
DEEPGRAM_API_KEY = '2e3d8166e27ee93b7ce2c3bd78a3beaa0f432d0e'
deepgram = DeepgramClient(DEEPGRAM_API_KEY)

# JavaScript for recording audio
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader();
  reader.onloadend = () => resolve(reader.result);
  reader.readAsDataURL(blob);
});
var record = time => new Promise(async resolve => {
  try {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    recorder = new MediaRecorder(stream);
    chunks = [];
    recorder.ondataavailable = e => chunks.push(e.data);
    recorder.start();
    await sleep(time);
    recorder.onstop = async () => {
      blob = new Blob(chunks, { type: 'audio/wav' });
      text = await b2text(blob);
      resolve(text);
    };
    recorder.stop();
  } catch (err) {
    console.error('Error accessing microphone:', err);
    resolve('error: ' + err.message);
  }
});
"""

# Record from mic
def record(seconds=5, filename='recorded_audio.wav'):
    print(f"Recording for {seconds} seconds...")
    display(Javascript(RECORD))
    js_result = output.eval_js(f'record({seconds * 1000})')

    if js_result.startswith('error:'):
        print(f"Recording failed: {js_result}")
        return upload_audio()

    audio_data = js_result.split(',')[1]
    audio_bytes = base64.b64decode(audio_data)

    with open(filename, 'wb') as f:
        f.write(audio_bytes)

    print(f"Audio saved as {filename}")
    return filename

# Upload fallback
def upload_audio():
    print("Please upload an audio file (WAV/MP3)...")
    uploaded = files.upload()
    if not uploaded:
        return None
    audio_file = list(uploaded.keys())[0]
    print(f"Uploaded file: {audio_file}")
    return audio_file

# Transcription via Deepgram
def transcribe_audio(audio_file):
    if not audio_file:
        return None
    print("Transcribing...")
    with open(audio_file, 'rb') as f:
        audio_data = f.read()
    options = PrerecordedOptions(model="nova-2", language="en", smart_format=True, punctuate=True)
    response = deepgram.listen.prerecorded.v("1").transcribe_file({"buffer": audio_data}, options)
    text = response['results']['channels'][0]['alternatives'][0]['transcript']
    print(f"Transcript: {text}")
    return text

# TTS via Deepgram
def text_to_speech(text, output_file="output_audio.mp3"):
    if not text:
        return None
    print("Converting to speech...")
    options = SpeakOptions(model="aura-asteria-en", encoding="mp3")
    deepgram.speak.rest.v("1").save(output_file, {"text": text}, options)
    print(f"Speech saved as {output_file}")
    return output_file

# Use updated LangChain OllamaLLM
llm = OllamaLLM(model="phi4")  # Add base_url="http://localhost:11434" if running remotely

# Get LLM response
def process_with_phi4(text):
    if not text:
        return "Sorry, I couldn't understand that."
    print("Querying Phi-4 LLM...")
    try:
        response = llm.invoke(text)
        print(f"LLM Response: {response}")
        return response
    except Exception as e:
        print(f"LLM error: {e}")
        return "Error generating response."

# Main Workflow
def main():
    audio_file = record(seconds=5)
    transcribed_text = transcribe_audio(audio_file)
    response_text = process_with_phi4(transcribed_text)
    output_audio = text_to_speech(response_text)
    if output_audio:
        print("Playing response...")
        display(Audio(filename=output_audio, autoplay=True))
        print("Download files:")
        if audio_file:
            files.download(audio_file)
        files.download(output_audio)

# Run
main()


Recording for 5 seconds...


<IPython.core.display.Javascript object>

Audio saved as recorded_audio.wav
Transcribing...


<ipython-input-17-544e01c2be2b>:90: UnsupportedWarning: prerecorded is unsupported as of 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  response = deepgram.listen.prerecorded.v("1").transcribe_file({"buffer": audio_data}, options)


Transcript: What is the capital of Bangladesh?
Querying Phi-4 LLM...
LLM Response: The capital of Bangladesh is Dhaka.
Converting to speech...
Speech saved as output_audio.mp3
Playing response...


Download files:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ollama run phi4


Error: llama runner process has terminated: exit status 2


In [ ]:
!nohup ollama serve > output.log 2>&1 &